In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import numpy
import copy
import qiskit
import ipyparallel as ipp
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit.providers.aer import AerSimulator
from openfermion.linalg import givens_decomposition_square as givensRotns
from qiskit.opflow import I,X,Y,Z,StateFn,AbelianGrouper
from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit_nature.mappers.second_quantization import JordanWignerMapper,ParityMapper
from qiskit_nature.problems.second_quantization.electronic.builders import fermionic_op_builder
from qiskit.quantum_info import Pauli
from qiskit.chemistry import FermionicOperator
from qiskit.aqua.operators.legacy import op_converter
from openfermion.circuits import slater_determinant_preparation_circuit
from qiskit import QuantumCircuit,execute,QuantumRegister,ClassicalRegister
from qiskit import Aer
from qiskit.circuit.random import random_circuit
from joblib import Parallel,delayed
from qiskit.aqua.operators import WeightedPauliOperator,Z2Symmetries
from qiskit.chemistry.components.variational_forms import UCCSD
import scipy

In [49]:
def countYgates(pauli_label):
    countYgates = sum(map(lambda x : 1 if 'Y' in x else 0, pauli_label))
    return countYgates
def egBandHamiltonianPartition(U):
    #Getting chemical Potential for Half-Filling
    with open('../chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('../'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1_0=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1_1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                line=line.split()
                i,j=map(int,line[:2])
                val=float(line[2])
                if(i==j):
                    eg_h1_0[i,j]=eg_h1_0[i+num_sites,j+num_sites]=val
                else:
                    eg_h1_1[i,j]=eg_h1_1[i+num_sites,j+num_sites]=val
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1_0=eg_h1_0+chem_pot       
    with open('../'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2_0=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        eg_h2_1=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            if((i==j) and (k==l)):
                eg_h2_0[i,j,k,l] = eg_h2_0[i+num_sites,j+num_sites,k,l] = eg_h2_0[i,j,k+num_sites,l+num_sites] \
                = eg_h2_0[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
            else:
                eg_h2_0[i,j,k,l] = eg_h2_0[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
                eg_h2_1[i+num_sites,j+num_sites,k,l] = eg_h2_1[i,j,k+num_sites,l+num_sites] = 0.5*val
    return eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1

def qubitOp(h1,h2):  
    fer_op=fermionic_op_builder.build_ferm_op_from_ints(h1,h2)
    qubit_conv = QubitConverter(ParityMapper(), two_qubit_reduction=True, z2symmetry_reduction="auto")
    qubit_op = qubit_conv.convert(fer_op, len(h1)//2)
    #mapper=JordanWignerMapper()
    #qubit_op=mapper.map(fer_op)
    return qubit_op 

def LRF_2_body_OD_terms(h2): #For 2 body off-diagonal terms
    def reshape_vec_to_mat(eigs):
        weight,vec1,vec2=eigs
        num_qubits=int(numpy.sqrt(vec1.shape[0]))
        L1=numpy.reshape(vec1,(num_qubits,num_qubits))
        L2=numpy.reshape(vec2,(num_qubits,num_qubits))
        return [numpy.sqrt(weight)*L1,numpy.sqrt(weight)*L2]
    #checksum  
    def higherRankRep(L):
        T=numpy.zeros((num_qubits,num_qubits,num_qubits,num_qubits))*1j
        for p in range(num_qubits):
            for q in range(num_qubits):
                for r in range(num_qubits):
                    for s in range(num_qubits):
                        T[p,q,r,s]=L[p,q]*L[r,s]
                        
        return T
    #constructing Low rank factorization circuits
    #Super matrix rep of four rank tensor (NxNxNxN)->(N^2xN^2)
    num_qubits=len(h2)
    N2=num_qubits*num_qubits
    h2_pq_rs=numpy.reshape(h2,(N2,N2))
    #Diagonalize Super matrix
    u,d,v=numpy.linalg.svd(h2_pq_rs,full_matrices=True)
    #Build Cholesky vectors
    Larr=[]
    arr=[(d[i],u[:,i],v[i]) for i in range(len(v)) if d[i]>1e-4]
    Larr=list(map(reshape_vec_to_mat,arr))
    #print(Larr)
    Larr=[Larr[i][0] for i in range(len(Larr))]
    qubit_h2=qubitOp(numpy.zeros((8,8)),h2)
    num_qubits_red=qubit_h2.num_qubits
    U1=PauliSumOp.from_list([('I'*num_qubits_red,numpy.cos(numpy.pi/4)),('IIIIIY',-1j*numpy.sin(numpy.pi/4))])
    U2=PauliSumOp.from_list([('I'*num_qubits_red,numpy.cos(numpy.pi/4)),('IIYIII',-1j*numpy.sin(numpy.pi/4))])
    rotH=(U2@U1@qubit_h2@U1.adjoint()@U2.adjoint()).reduce()
    qc=QuantumCircuit(num_qubits_red)
    qc=add_multiqubit_gate('IIIIIY',numpy.pi/4,qc)
    qc=add_multiqubit_gate('IIYIII',numpy.pi/4,qc)
    return qc,rotH,qubit_h2

def add_multiqubit_gate(pauli_string, param, circuit):
    num_qubits=circuit.num_qubits
    qr=QuantumRegister(num_qubits,'q')
    if pauli_string == 'I'*num_qubits:
        gate = 1
        for j in range(len(pauli_string)):
            gate = numpy.kron(gate, Pauli('I').to_matrix())
        gate *= -1j * np.sin(param)
        gate += numpy.cos(param) * np.eye(2**num_qubits)
        circuit.unitary(gate, qr, label=pauli_string)
    else:
        qubits_to_act_on = []
        gate = 1
        for j in range(len(pauli_string)):
            if pauli_string[j] != 'I':
                gate=numpy.kron(Pauli(pauli_string[j]).to_matrix(),gate)
                qubits_to_act_on.append(num_qubits-j-1)
        gate *= (-1j * numpy.sin(param))
        gate += numpy.cos(param) * numpy.eye(2**len(qubits_to_act_on))
        List_regs=[qr[i] for i in qubits_to_act_on]#[::-1]
        Label=pauli_string+"\n{:0.02f}".format(param.real)
        circuit.unitary(gate, List_regs, label = Label)
    return circuit  

def get_givens_rotns(h):
        num_qubits=len(h)
        qubitH=qubitOp(h,numpy.zeros((num_qubits,num_qubits,num_qubits,num_qubits)))
        num_qubits_red=qubitH.num_qubits
        H=qubitH.to_matrix()
        w,v=numpy.linalg.eigh(h)
        givens=givensRotns(v.T)[0]
        rotH=qubitH
        circ=QuantumCircuit(num_qubits_red)
        for rots in givens:
            for tup in rots:
                i,j,theta,phi=tup
                h1=numpy.zeros((num_qubits,num_qubits))*1j
                h1[i,j]=-1j
                h1[j,i]=1j
                qubit_h1=qubitOp(h1,numpy.zeros((num_qubits,num_qubits,num_qubits,num_qubits)))
                List=qubit_h1.primitive.to_list()
                Op=PauliSumOp.from_list([('I'*num_qubits_red,1)])
                for i in range(len(List)):
                    Op=PauliSumOp.from_list([('I'*num_qubits_red,numpy.cos(theta*List[i][1])),(List[i][0],1j*numpy.sin(theta*List[i][1]))])@Op
                    circ=add_multiqubit_gate(List[i][0], theta*List[i][1], circ)
                rotH=Op@rotH@Op.adjoint()
                rotH=rotH.reduce().reduce()
        return circ,rotH,qubitH
    
def doubly_decomposed_form(h1_D,h1_X,h2_D,h2_X):

    #getting the givens rotation circuit and the rotated Hamiltonian for the one body terms
    circ_1_body,rotH_1_X,qubitH_1_X=get_givens_rotns(h1_X)
    #getting Cholesky vectors for two body off-diag terms
    circ_2_body,rotH_2_X,qubitH_2_X=LRF_2_body_OD_terms(h2_X)
    qubitH_D=qubitOp(h1_D,h2_D)
    circs_and_rotH_arr=[(circ_1_body,rotH_1_X,qubitH_1_X),(circ_2_body,rotH_2_X,qubitH_2_X),(qubitH_D)]
    return circs_and_rotH_arr

def expectation(h,circ):
    state=execute(circ,Aer.get_backend('statevector_simulator'),shots=1024).result().get_statevector()
    h_expec=state.conj()@h.to_matrix()@state
    return h_expec


def hamiltonian_expectation_in_doubly_decomposed_form_sv(params):
    
    ansatz_circ=var_form_base.construct_circuit(params)
    circ_rot_Arr=[ansatz_circ.copy(),ansatz_circ.copy(),ansatz_circ.copy()]
    h_part_expectation_arr=[]
    circ_rot_Arr[0]=circ_rot_Arr[0]+circs_and_rotH[0][0]
    h_part_expectation_arr.append(expectation(circs_and_rotH[0][1],circ_rot_Arr[0]))
    circ_rot_Arr[1]=circ_rot_Arr[1]+circs_and_rotH[1][0]
    h_part_expectation_arr.append(expectation(circs_and_rotH[1][1],circ_rot_Arr[1]))
    h_part_expectation_arr.append(expectation(circs_and_rotH[2],circ_rot_Arr[2]))
    h_sum_DD=sum(h_part_expectation_arr).real
    qubitH=qubitOp(h1_D+h1_X,h2_X+h2_D)
    h_sum_direct=expectation(qubitH,ansatz_circ).real
    print("doubly decomposed",h_sum_DD)
    print("direct",h_sum_direct)      
    return h_sum_DD    

def hamiltonian_estimation_in_doubly_decomposed_form_qasm(input_vars):
    
    def Energy_Estimator_DD(input_vars):
        
        def getCounts(Input):
            circ,num_shots=Input
            circ1=circ.copy()
            circ1.add_register(ClassicalRegister(6,'c'))
            circ1.measure([0,1,2,3,4,5],[0,1,2,3,4,5]) 
            job_sim = execute(circ1, AerSimulator(method='density_matrix'),shots=num_shots) 
            result_sim = job_sim.result()
            #density_matrix = result_sim
            counts = result_sim.get_counts(circ1) 
            return counts#,density_matrix
        def estimate(PauliOp,counts):
            PauliString,coeff=PauliOp.primitive.to_list()[0]
            indexOfZs=numpy.where(numpy.array(list(PauliString))=='Z')[0]
            bitstrings,cnts=list(counts. keys()),list(counts.values())
            p=0
            count_valid=0
            for i in range(len(bitstrings)):
                bit_Str_to_Arr=numpy.array(list(bitstrings[i]))
                #flag1=1 if len(numpy.where(bit_Str_to_Arr=='1')[0])==4 else 0 #check number of electrons
                #flag2=1 if len(numpy.where(bit_Str_to_Arr[0:4]=='1')[0])==2 else 0 #check net spin
                #if ((flag1==1) and (flag2==1)):
                #count_valid=count_valid+cnts[i]
                counter_1=list(numpy.array(list(bitstrings[i]))[indexOfZs]).count('1')
                if counter_1%2==1:
                    p=p+cnts[i]
            p=p/sum(cnts)
            mean=(1-2*p)
            return mean*coeff.real
        def meanOp(PauliOp,counts):
            mean=0
            for i in range(len(PauliOp)):
                mean=mean+estimate(PauliOp[i],counts)
            return mean 
        def covariance(Op,i,j,counts):
            cov=estimate(Op[i]@Op[j],counts)-estimate(Op[i],counts)*estimate(Op[j],counts)
            return cov
        def covbtnOps(A,B,counts):
            corr=0
            for i in range(len(A)):
                for j in range(len(B)):
                    corr=corr+estimate(A[i]@B[j],counts)-estimate(A[i],counts)*estimate(B[j],counts)
            return corr        
        def variance(Op,counts):
            var=0
            cov_mat=numpy.zeros((len(Op),len(Op)))
            for i in range(len(Op)):
                for j in range(i,len(Op)):
                    if i==j:
                        mean=estimate(Op[i],counts)
                        coeff=Op[i].primitive.to_list()[0][1].real
                        cov_mat[i][j]=(coeff**2-mean**2)
                        var=var+(coeff**2-mean**2)
                    elif i!=j:
                        var=var+2*covariance(Op,i,j,counts)
                        cov_mat[i][j]=cov_mat[j][i]=covariance(Op,i,j,counts)
            return var
        def OpErr(data):
            Op,counts=data
            var=variance(Op,counts)
            print("variance",var)
            err=numpy.sqrt(var/sum(counts.values()))
            return err.real
        circ,Op,num_shots=input_vars
        counts_Arr=getCounts((circ,num_shots))
        m1=meanOp(Op,counts_Arr)
        err1=OpErr((Op,counts_Arr))
        return m1,err1
    params,num_shots_arr=input_vars
    ansatz_circ=var_form_base.construct_circuit(params)
    circ_rot_Arr=[ansatz_circ.copy(),ansatz_circ.copy(),ansatz_circ.copy()]
    h_part_expectation_arr=[]
    circ_rot_Arr[0]=circ_rot_Arr[0]+circs_and_rotH[0][0]
    circ_rot_Arr[1]=circ_rot_Arr[1]+circs_and_rotH[1][0]
    #Estimate from statistics
    data_arr=[(circ_rot_Arr[0],circs_and_rotH[0][1],num_shots_arr[0]),
              (circ_rot_Arr[1],circs_and_rotH[1][1],num_shots_arr[1]),
              (circ_rot_Arr[2],circs_and_rotH[2],num_shots_arr[2])]
    results=Parallel(n_jobs=6,verbose=2)(delayed(Energy_Estimator_DD)(data_arr[i]) for i in range(len(data_arr)))
    m1,err1=results[0][0],results[0][1]
    m2,err2=results[1][0],results[1][1]
    m3,err3=results[2][0],results[2][1]
    print("total mean",m1+m2+m3,"total error",err1+err2+err3,"ind means",m1,m2,m3,"ind errs",err1,err2,err3)
    return m1+m2+m3#,err1+err2+err3,m1,m2,m3,err1,err2,err3  
def Energy(params):
    circ=var_form_base.construct_circuit(parameters=params)
    state=execute(circ,Aer.get_backend('statevector_simulator'),shots=1024).result().get_statevector()
    E=(state.conj()@Hmat@state).real
    return E

In [50]:
def commutator(A,B):
        B2=WeightedPauliOperator([[-B.paulis[0][0],B.paulis[0][1]]])
        return A.multiply(B).add(B2.multiply(A))
def commutatorsForGradient(Op):
    Op_conj=WeightedPauliOperator([[-1j,Op.paulis[0][1]]])
    M=Op.multiply(qubitH).add(qubitH.multiply(Op_conj))
    Mmat=op_converter.to_matrix_operator(M).matrix
    return Mmat
def commutatorPool(qubitH,stripZs=False):
    def stripZ(label):
        label=numpy.array(list(label))
        label[label=='Z']='I'
        label=''.join(list(label))
        return label
    #construct commutator pool from the Hamiltonian
    pool_H=[WeightedPauliOperator([[1j,qubitH.paulis[i][1]]]) for i in range(len(qubitH.paulis))][1:]
    #commutator between operators
    commutator_pool=[WeightedPauliOperator([[1j,commutator(op1,op2).paulis[0][1]]]) for op1 in pool_H for op2 in pool_H if countYgates(commutator(op1,op2).paulis[0][1].to_label())%2==1]
    labels=[commutator_pool[i].paulis[0][1].to_label() for i in range(len(commutator_pool))]
    unique_labels=numpy.unique(labels)
    if stripZs==True:
        new_labels=[]
        for i in range(len(unique_labels)):
            new_labels.append(stripZ(unique_labels[i]))
        unique_labels=numpy.unique(new_labels)    
    commutator_pool=[WeightedPauliOperator([[1j,Pauli(unique_labels[i])]]) for i in range(len(unique_labels))]
    return commutator_pool
def compute_gradient(M,state):
    grad=state@M@numpy.conjugate(state)
    return grad.real

In [51]:
#SMO with qasm
def SMO_qasm(cost,params,runs=20,tol=1e-4,save_opt_steps=False):
    index=1
    conv_err=1000
    t1=time.time()
    def Energy(params):
        backend = Aer.get_backend('statevector_simulator')
        circ=var_form_base.construct_circuit(parameters=params)
        stateVector_0=execute(circ,backend,shots=1024).result().get_statevector()
        E=numpy.real(numpy.dot(numpy.dot(numpy.conjugate(stateVector_0),Hmat),stateVector_0)) 
        return E
    def E_landscape(ind,ang,cost,params):
        params1=copy.deepcopy(params)
        params1[ind]=params1[ind]+ang #ang
        data=cost((params1,num_shots_arr))
        return data
    def determine_unknowns(E,cost,params,ind):
        L1=E_landscape(ind,0,cost,params)
        L2=E_landscape(ind,numpy.pi/4.,cost,params)
        L3=E_landscape(ind,-numpy.pi/4.,cost,params)
        ratio=(L3-L2)/(2*L1-L2-L3)
        a3=(L2+L3)/2.
        a2=2*params[ind]-numpy.arctan(ratio)
        a1=(L1-a3)/numpy.cos(numpy.arctan(ratio))
        return a1,a2,a3
    def update(E,cost,params,ind):
        t1=time.time()
        a1,a2,a3=determine_unknowns(E,cost,params,ind)
        thetaStar=a2/2.+numpy.pi/2. if a1>0 else a2/2.
        newParams=copy.deepcopy(params)
        newParams[ind]=thetaStar
        updEnergy=cost((newParams,num_shots_arr))
        #print("time taken",time.time()-t1)
        return newParams,updEnergy
    while conv_err>tol and index<runs:
        print("looped "+str(index)+" times")
        Eold=cost((params,num_shots_arr))
        init=0
        for i in range(len(params)):#[::-1][0:1]:
            #first run sequential minimial optimization (SMO)  for a given multiqubit operator using 
            #exact analytical form for cost function
            if init==0:
                E=Eold
            ind=i
            params,E=update(E,cost,params,ind)
            print("upd_energy",Energy(params))
            if save_opt_steps==True:
                with open('adaDeltaOptStepsWithQasm_parity.txt','a') as f:
                    Str=["{:0.16f}".format(params[i]) for i in range(len(params))]
                    #Str2=["{:0.16f}".format(elem) for elem in [E,MSE,Energy(params)]]
                    print('['+','.join(Str)+']'+'#'+"{:0.16f}".format(Energy(params)),file=f) 
            else:
                continue
            init=init+1  
        conv_err=numpy.abs(Eold-E)
        print("inner loop error",conv_err)
        index=index+1
    res={'x':params,'fun':E}        
    return res
def AdaDelta_qasm(cost,params,runs=20,tol=1e-4,num_shots_arr=[2**8,2**9,2**9],save_opt_steps=False):
    string_shots_label=','.join([str(shots) for shots in num_shots_arr])
    v=numpy.zeros((len(params)))*0.0
    delta=numpy.zeros((len(params)))*0.0
    beta=0.9
    rho=0.8
    directions= numpy.arange(len(params))
    delta=0
    conv_err=1
    old_conv_err=1
    Eold=cost((params,num_shots_arr))
    E_arr=[]
    ind=0
    eps=numpy.array([5e-7]*len(params))#1e-5,5e-6,1e-7,5e-8
    Max=1e5
    lr0=0.02
    def learningRateEnv(lr0,runs):
        lr_u=lr0+Max*numpy.exp(-(33/runs)*numpy.linspace(0,runs,runs+1))
        lr_d=lr0*(1-numpy.exp(-(33/runs)*numpy.linspace(0,runs,runs+1)))
        return lr_u,lr_d
    def Energy(params):
        backend = Aer.get_backend('statevector_simulator')
        circ=var_form_base.construct_circuit(parameters=params)
        stateVector_0=execute(circ,backend,shots=1024).result().get_statevector()
        E=numpy.real(numpy.dot(numpy.dot(numpy.conjugate(stateVector_0),Hmat),stateVector_0)) 
        return E
    def StochGrad(cost,params,directions,num_shots_arr):
        shiftedParams1=[(numpy.array(params)+numpy.array([0]*ei+[numpy.pi/4.]+[0]*(len(params)-ei-1)),num_shots_arr) for ei in directions]
        shiftedParams2=[(numpy.array(params)+numpy.array([0]*ei+[-numpy.pi/4.]+[0]*(len(params)-ei-1)),num_shots_arr) for ei in directions]
        stochGrads=numpy.zeros((len(params)))
        estm1Arr=map(cost,shiftedParams1)
        estm2Arr=map(cost,shiftedParams2)
        stochGrads=0.5*(numpy.array(list(estm1Arr))-numpy.array(list(estm2Arr)))
        return stochGrads
    lr_u,lr_d= learningRateEnv(lr0,runs)
    flag=0
    flag1=0
    terminate=False
    while ((conv_err>tol) and (ind<runs)):
        print("index",ind)
        old_conv_err=conv_err
        g_stoch=StochGrad(cost,params,directions,num_shots_arr)
        v = beta*v+(1-beta)*g_stoch*g_stoch
        lr=numpy.sqrt(delta+eps)/numpy.sqrt(v+eps)  #learning rate
        #clipping learning rates, increasing shots and restarting ada delta
        if ((conv_err<5e-4) and (flag==0)):
            flag=1
            num_shots_arr=[2**17,2**19,2**19]
            print("************shots increased**************")
        if ((conv_err<4e-5) and (flag1==0)):
            flag1=1
            num_shots_arr=[2**17,2**19,2**20]
            print("************shots increased**************")    
        for i in range(len(lr)):
            if lr[i]>lr_u[ind]:
                eps[i]=lr_u[ind]*lr_u[ind]*v[i]
                lr[i]=lr_u[ind]
                delta[i]=0
            elif lr[i]<lr_d[ind]:
                eps[i]=lr_d[ind]*lr_d[ind]*v[i]
                lr[i]=lr_d[ind]
                delta[i]=0
            else:
                continue
        g=lr*g_stoch
        params=params-g
        params=params.real
        delta= beta*delta+(1-beta)*g*g
        E=Energy(params)#HamiltonianEstm(params)[0]
        E_arr.append(E)
        print("updated energy",E,"learning rate",lr)#,Energy(params))
        if save_opt_steps==True:
            with open('adaDeltaOptStepsWithQasm_parity.txt','a') as f:
                print('['+','.join([str(params[i]) for i in range(len(params))])+']'+'#'+str(E),file=f)
        ind=ind+1 
        conv_err=rho*conv_err+(1-rho)*numpy.abs(Eold-E)
        print("error",conv_err)
        Eold=E
    res={'x':params,'fun':E}    
    return res

In [52]:
U=7
h1_D,h1_X,h2_D,h2_X=egBandHamiltonianPartition(U)
circs_and_rotH=doubly_decomposed_form(h1_D,h1_X,h2_D,h2_X)    

In [53]:
#preparing commutator pool
def qubitWeightedOp(h1,h2):
    qubit_op=FermionicOperator(h1,h2).mapping('parity')
    qubitH=Z2Symmetries.two_qubit_reduction(qubit_op, qubit_op.num_qubits//2)
    return qubitH
qubitH=qubitWeightedOp(h1_D+h1_X,h2_D+h2_X)
Hmat=op_converter.to_matrix_operator(qubitH).dense_matrix
w,v=numpy.linalg.eigh(Hmat)
Eg=w[0]
commutator_pool=commutatorPool(qubitH,stripZs=True)
#constructing matrices for gradient computation
MatrixRepOfPoolOps=list(map(commutatorsForGradient,commutator_pool))
print("pool sizes",len(MatrixRepOfPoolOps))
backend=Aer.get_backend('statevector_simulator')
#preparing init state
circ=QuantumCircuit(6)
circ.x(0)
circ.x(3)
var_form_base=UCCSD(8,num_particles=4, initial_state=circ,qubit_mapping='parity',two_qubit_reduction=True)
var_form_base.manage_hopping_operators()
error=1000

pool sizes 16


In [54]:
import time

In [55]:
with open('adaDeltaOptStepsWithQasm_parity.txt','r') as f:
    lines=f.readlines()
    Labels=[]
    Energy_arr=[]
    for i in range(len(lines)):
        if lines[i][0]=='l':
            Labels.append(lines[i].split('-')[1][1:-1])
            if i!=0:
                
                Energy_arr.append(eval(lines[i-1].split('#')[1]))
    Energy_arr.append(eval(lines[-1].split('#')[1]))            
    params=eval(lines[-1])        

In [56]:
Energy_arr, Labels

([-18.672676419750214,
  -18.939408985421256,
  -18.95962017471545,
  -18.979117950408792,
  -19.004011977874764,
  -19.005727026117718,
  -19.007318606049633,
  -19.00740591815112,
  -19.007396572109688,
  -19.00738928548016,
  -19.008172556949944,
  -19.023562463227375,
  -19.028009387091263,
  -19.035340099116354,
  -19.04051821356684],
 ['YXIIII',
  'IYXIII',
  'IIIYXI',
  'IIIIYX',
  'YXIIII',
  'IIXYXX',
  'IIXIYI',
  'IIXIIY',
  'YXXIIX',
  'IYIIIX',
  'IIYIIX',
  'IIXIYI',
  'IXYIII',
  'IIXIYI',
  'YXXIIX'])

In [57]:
for i in range(len(Labels)):
    var_form_base.push_hopping_operator(WeightedPauliOperator([[1j,Pauli(Labels[i])]]))

In [58]:
Energy(params)

-19.04051821356684

In [65]:
hamiltonian_estimation_in_doubly_decomposed_form_qasm((params,[2**19,2**19,2**20]))

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


total mean -19.040029894146507 total error 0.0017669768535109797 ind means -0.7248554274959862 -0.03626518249511717 -18.278909284155404 ind errs 0.0006125446371886889 0.0004933620237148015 0.0006610701926074893


[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    3.2s finished


-19.040029894146507

In [ ]:
num_shots_arr=[2**19,2**19,2**20]
res=SMO_qasm(hamiltonian_estimation_in_doubly_decomposed_form_qasm,params,runs=20,tol=1e-4,save_opt_steps=True)

looped 1 times


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


total mean -19.040517996495787 total error 0.0017676287657054442 ind means -0.7247932944665519 -0.03657760620117186 -18.279147095828062 ind errs 0.0006127561565163349 0.000492829965365703 0.0006620426438234062


[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    3.3s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


total mean -19.04061911208163 total error 0.0017673910545000148 ind means -0.7248381683211433 -0.03698215484619139 -18.278798788914294 ind errs 0.0006115550830978204 0.0004939592993782707 0.0006618766720239239


...

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


total mean -19.04152813607824 total error 0.0017651498330866781 ind means -0.7255009206351081 -0.03699817657470701 -18.279029038868426 ind errs 0.0006119329223718605 0.0004935345965843499 0.0006596823141304679


[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.7s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


total mean -19.04162508371256 total error 0.001770443512939104 ind means -0.7256412952571633 -0.03763504028320311 -18.278348748172196 ind errs 0.000611621869505129 0.000496182515517998 0.000662639127915977


[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    3.9s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


In [118]:
#res=AdaDelta_qasm(hamiltonian_estimation_in_doubly_decomposed_form_qasm,params,runs=50,tol=1e-6,num_shots_arr=[2**18,2**19,2**19],save_opt_steps=True)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


index 0


[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:   11.4s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:   11.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.6s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.5s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.7s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.4s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.7s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concu

updated energy -19.040540548437136 learning rate [0.64688204 0.88037219 0.91731048 0.56480451 0.81870449 0.83163163
 0.52045962 0.14040494 0.11240277 0.18949385 0.99308    0.73685136
 0.6616828  0.51776987 0.42954412]
error 0.800230209888553
index 1


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.5s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.6s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.6s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.6s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.3s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.5s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:   

updated energy -19.040341136503486 learning rate [0.7793214  0.80525646 0.54583194 0.39852603 0.48007352 0.45613224
 0.16902075 0.1984264  0.0810042  0.09118528 0.14755905 0.10974084
 0.53845821 0.15137234 0.42662082]
error 0.6402240502975725
index 2


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.5s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.2s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.8s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.5s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.4s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.5s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:   

updated energy -19.040178841683847 learning rate [0.82021294 0.28464053 0.53891763 0.1554787  0.25868867 0.56808519
 0.09108015 0.08991276 0.10898331 0.10931121 0.0671905  0.10140439
 0.68005275 0.08116618 0.07800539]
error 0.5122116992019856
index 3


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.4s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.4s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.2s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.4s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.4s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.3s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:   

KeyboardInterrupt: 

In [101]:
#params=[]
EnergyArr=[]
ExcOps=[]
ti=time.time()
steps=35
for i in range(35):
    save_steps=False
    circ=var_form_base.construct_circuit(parameters=params)
    state=execute(circ,backend,shots=1024).result().get_statevector()
    grads=numpy.array(Parallel(n_jobs=7,verbose=2)(delayed(compute_gradient)(MatrixRepOfPoolOps[i],state) for i in range(len(commutator_pool))))#qubit_pool_UCCSD))))#
    indexes=numpy.argsort(abs(grads))[::-1][:3]
    print("three highest grads",grads[indexes[0]],grads[indexes[1]],grads[indexes[2]])
    if len(ExcOps)!=0:
        if (commutator_pool[indexes[0]].paulis[0][1].to_label()!=ExcOps[-1]):
            PauliOp=commutator_pool[indexes[0]]
        else:
            PauliOp=commutator_pool[indexes[1]]
    else:
        PauliOp=commutator_pool[indexes[0]]
    ExcOps.append(PauliOp.paulis[0][1].to_label())
    print("chosen Op",ExcOps[-1])
    with open('adaDeltaOptStepsWithQasm_parity.txt','a') as f:
        print("label-",ExcOps[-1],file=f)
    params.append(0.0)
    var_form_base.push_hopping_operator(PauliOp)
    #construct circuit for estimating Hamiltonian
#     circ=var_form_base.construct_circuit(parameters=params)
    res=AdaDelta_qasm(hamiltonian_estimation_in_doubly_decomposed_form_qasm,params,runs=300,tol=1e-6,num_shots_arr=[2**17,2**18,2**19],save_opt_steps=True)
    E,params=res['fun'],list(res['x'])
    save_steps=True
    #res=scipy.optimize.minimize(Energy_Estimator,params,method='BFGS',options={'ftol':1e-7})
    #params,E=list(res['x']),res['fun']
    print("num_params",var_form_base.num_parameters)
    print("Energy",E)
    EnergyArr.append(E)
    print("time elapsed",time.time()-ti)    
    error=EnergyArr[-1]-Eg

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


three highest grads -0.08099661180207375 0.06087300896805441 -0.030229557061913454
chosen Op IIXIYI


[Parallel(n_jobs=7)]: Done  12 out of  16 | elapsed:    1.0s remaining:    0.3s
[Parallel(n_jobs=7)]: Done  16 out of  16 | elapsed:    1.0s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


index 0


[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    6.0s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    5.5s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    2.0s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    2.3s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    1.9s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    2.0s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    2.0s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concu

...

updated energy -19.03607309209017 learning rate [0.66327242 0.17427582 0.29807171 0.15430316 0.20494802 0.24233125
 0.09570174 0.06466001 0.07121186 0.06114964 0.06658076 0.09081107
 0.40963887 0.08230357 0.0399711 ]
error 0.00018127173652902627
index 74


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    2.3s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    2.4s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    2.3s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    2.4s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    2.4s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    2.4s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:   

updated energy -19.036064953415536 learning rate [0.68284399 0.16908804 0.25503505 0.1595352  0.20258422 0.24371314
 0.09730083 0.06425485 0.07041668 0.06118657 0.06688508 0.0886641
 0.47351483 0.08213697 0.04228616]
error 0.00014664512414997783
index 75


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    2.8s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    3.0s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    3.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    2.7s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    2.6s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    2.4s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:   

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.